In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import cv2
import os
import seaborn as sns
import sklearn
import zipfile
import random
import math
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.utils.data_utils import Sequence
import albumentations

In [ ]:
keypoint_data = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv')
keypoint_data.head(10).T

# **Unzip File**

In [ ]:
destination = '../kaggle/working/'

train_archive = zipfile.ZipFile('../input/facial-keypoints-detection/training.zip')
test_archive = zipfile.ZipFile('../input/facial-keypoints-detection/test.zip')



def extraction(archive, destination):
    for file in archive.namelist():
        archive.extract(file, destination)
        
        
extraction(train_archive, destination)
extraction(test_archive, destination)

# **Or use !unzip**

In [ ]:
!unzip -n ../input/facial-keypoints-detection/test.zip
!unzip -n ../input/facial-keypoints-detection/training.zip

In [ ]:
TRAIN_DIR = './training.csv'
TEST_DIR = './test.csv'
LOOKID_DIR = '../input/facial-keypoints-detection/IdLookupTable.csv'


In [ ]:
train_data = pd.read_csv(TRAIN_DIR)
test_data = pd.read_csv(TEST_DIR)

## **Lets look at the train data**

In [ ]:
train_data.head(10).T

In [ ]:
train_data.info()

In [ ]:
train_data.isnull().sum()

## **Lets see the test data**

In [ ]:
test_data.head()

## **LOOK ID Directory**

In [ ]:
lookup_data = pd.read_csv(LOOKID_DIR)
lookup_data.head()

### Pairplot

In [ ]:
sns.pairplot(train_data)
plt.savefig('pairplot.png', dpi=300)

### Histogram Plot

In [ ]:
fig, ax = plt.subplots(6, 5, figsize=(15, 10))
ax = ax.ravel() 
for i in range(30):
    ax[i].hist(train_data[train_data.columns[i]], bins=50, density=True, alpha=0.7, color='red')
    ax[i].set_title(train_data.columns[i],fontsize=10)
  # ax[i].axes.get_xaxis().set_visible(False)
plt.tight_layout()  
plt.savefig('hist_plot.png', dpi=200)

# **Handling Null Values with feature median values**

In [ ]:
train_data.fillna(train_data.describe().T['50%'], inplace= True)
train_data.head()

In [ ]:
train_data.isnull().sum()

## **Data Preprocessing**

In [ ]:
def load_images(image):
    images=[]
    for idx, sample in image.iterrows():
        img = np.array(sample['Image'].split(' '), dtype=np.float32)
        img = np.reshape(img, (96, 96,1))
        images.append(img)
        
    images = np.array(images)/255.0
    return images


def format_dataset(data):
    keypoints=[]
    features = data.drop('Image', axis=1)
    for idx, sample in features.iterrows():
        keypoints.append(sample)
    keypoints = np.array(keypoints, dtype='float')
    return keypoints



def sample(image, keypoint, axis=None, color='red'):
    if axis is None:
        fig, axis = plt.subplots()
        
    axis.scatter(keypoint[0::2], keypoint[1::2], s=8, c=color, marker='*')
    axis.imshow(image.squeeze(), cmap='gray')
    
    
        

def sample_images(image_data,keypoints, n_rows=6, n_cols=4):
    fig= plt.figure(figsize=(2*n_cols, 2*n_cols), dpi=200)
    
    for i, idx in enumerate(np.random.randint(0, len(keypoints), n_rows*n_cols)):
        ax = fig.add_subplot(n_rows, n_cols, i+1, xticks=[], yticks=[])
        sample(image_data[idx], keypoints[idx], axis=ax)
        

In [ ]:
images = load_images(train_data)
keypoints=  format_dataset(train_data)
print(images.shape)
print(keypoints.shape)

In [ ]:
test_images = load_images(test_data)
test_images.shape

## ***Show Sample Images***

In [ ]:

sample_images(images, keypoints)

# Different data augmentation

Inspired from this beautiful [Notebook](http://www.kaggle.com/balraj98/data-augmentation-for-facial-keypoint-detection) by Balraj Ashwath

### Rotation

In [ ]:
# Random angles
angles =[] 
for i in range(3):
    n= random.randint(10,20)
    angles.append(n)
    
    
def rotation(images, keypoints):
    rotated_images=[]
    rotated_keypoints=[]
    for i in angles:
        for angle in [i, -i]:
            rotate = cv2.getRotationMatrix2D((48, 48), angle, 1.0)
            radius = - angle*math.pi/180
            
            for image in images:
                image_rotation=  cv2.warpAffine(image, rotate, (96, 96), flags = cv2.INTER_CUBIC)
                #image_rotation=  np.reshape(image_rotation, ( 96, 96,1))
                rotated_images.append(np.reshape(image_rotation, (96, 96,1)))
                
                
            for keypoint in keypoints:
                keypoint = keypoint - 48
                
                for idx in range(0, len(keypoint), 2):
                    keypoint[idx]= keypoint[idx]*math.cos(radius)- keypoint[idx+1]*math.sin(radius)
                    
                    keypoint[idx+1]= keypoint[idx]*math.sin(radius)+ keypoint[idx+1]*math.cos(radius)
                    
                    
                keypoint =keypoint + 48
                    
                rotated_keypoints.append(keypoint)
                    
                    
    rotated_images = np.array(rotated_images)/255.0 
    rotated_keypoints = np.array(rotated_keypoints, dtype='float')
    return rotated_images, rotated_keypoints
                    
        
                    
rotated_images, rotated_keypoints = rotation(images, keypoints)

sample_images(rotated_images, rotated_keypoints)
print(rotated_images.shape) 
print(rotated_keypoints.shape)
            

In [ ]:
class data_process(Sequence):
    def __init__(self, image, keypoint, batch_size, augmentation):
        self.image = image
        self.keypoint = keypoint
        self.batch_size = batch_size
        self.augmentation = augmentation
        self.shuffle= True
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.ceil(len(self.image)/ float(self.batch_size)))
    
    
    def __getitem__(self, idx):
        indexes = self.indexes[idx*self.batch_size: (idx+1)*self.batch_size]
        
        batch_image = self.image[indexes, ...]
        batch_key = self.keypoint[indexes,:]
        
        if self.augmentation is not None:
            keypoints = np.array([tuple(zip(key[::2], key[1::2])) for key in batch_key])
            
            transformed_image = [self.augmentation(image=image, keypoints=keypoint) for image, keypoint in zip(batch_image, keypoints)]
            
            batch_image = np.stack([z['image'] for z in transformed_image], axis=0)
            
            batch_key = np.stack([np.array(z['keypoints']).flatten(order='C') for z in transformed_image], axis=0)
            
        return batch_image, batch_key
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image))
        if self.shuffle ==True:
            np.random.shuffle(self.indexes)
            
        

In [ ]:
xtrain, xval,ytrain, yval = train_test_split(images, keypoints, test_size =0.18, shuffle=True)

# MultiResUnet architecture

In [ ]:
def conv2d_block(input_tensor, filters, kernel_size, activation="relu"):
    
    
    x = keras.layers.Conv2D(filters, kernel_size=(kernel_size, kernel_size), 
                            padding='same', use_bias=False, strides=(1,1))(input_tensor)
    x = keras.layers.BatchNormalization(axis=3, scale=False)(x)
    if (activation==None):
        return x
        
    x = keras.layers.Activation(activation='relu')(x)

    return x


def MultiResBlock(filters, input_layers, alpha = 1.67):


    W = alpha * filters
    shortcut= input_layers
    shortcut = conv2d_block(shortcut, int(W*0.167) + int(W*0.333) +
                         int(W*0.5), 1,  activation=None)

    conv3x3 = conv2d_block(input_layers, int(W*0.167), 3, activation="relu" )

    conv5x5 = conv2d_block(conv3x3, int(W*0.333), 3, activation="relu")

    conv7x7 = conv2d_block(conv5x5, int(W*0.5), 3, activation="relu")

    out = keras.layers.concatenate([conv3x3, conv5x5, conv7x7], axis=3)
    out = keras.layers.BatchNormalization(axis=3)(out)

    out = keras.layers.add([shortcut, out])
    out = keras.layers.Activation('relu')(out)
    out = keras.layers.BatchNormalization(axis=3)(out)

    return out

In [ ]:
def ResPath(filters, length, input_layers):


    shortcut= input_layers
    shortcut = conv2d_block(shortcut, filters, 1,activation=None)

    out = conv2d_block(input_layers, filters, 3, activation="relu")

    out = keras.layers.add([shortcut, out])
    out = keras.layers.Activation('relu')(out)
    out = keras.layers.BatchNormalization(axis=3)(out)

    for i in range(length-1):

        shortcut = out
        shortcut = conv2d_block(shortcut, filters, 1, activation=None)

        out = conv2d_block(out, filters, 3, activation="relu")

        out = keras.layers.add([shortcut, out])
        out = keras.layers.Activation('relu')(out)
        out = keras.layers.BatchNormalization(axis=3)(out)

    return out

In [ ]:
def conv2d_block_T(input_tensor, filters, kernel_size):
    x = keras.layers.Conv2DTranspose(filters, kernel_size=(kernel_size, kernel_size), strides=(2, 2), padding="same")(input_tensor)
    x = keras.layers.BatchNormalization(axis=3, scale=False)(x)
    
    return x







In [ ]:



def MultiResUnet(input_shape):



    inputs = keras.Input(input_shape)

    mresblock1 = MultiResBlock(32, inputs)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(mresblock1)
    mresblock1 = ResPath(32, 4, mresblock1)

    mresblock2 = MultiResBlock(32*2, pool1)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(mresblock2)
    mresblock2 = ResPath(32*2, 3, mresblock2)

    mresblock3 = MultiResBlock(32*4, pool2)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(mresblock3)
    mresblock3 = ResPath(32*4, 2, mresblock3)

    mresblock4 = MultiResBlock(32*8, pool3)
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(mresblock4)
    mresblock4 = ResPath(32*8, 1, mresblock4)

    mresblock5 = MultiResBlock(32*16, pool4)

    up6 = keras.layers.concatenate([conv2d_block_T(mresblock5, 32*8, 2), mresblock4], axis=3)
    mresblock6 = MultiResBlock(32*8, up6)

    up7 = keras.layers.concatenate([conv2d_block_T(mresblock6, 32*4, 2), mresblock3], axis=3)
    mresblock7 = MultiResBlock(32*4, up7)

    up8 = keras.layers.concatenate([conv2d_block_T(mresblock7, 32*2, 2), mresblock2], axis=3)
    mresblock8 = MultiResBlock(32*2, up8)

    up9 = keras.layers.concatenate([conv2d_block_T(mresblock8, 32, 2), mresblock1], axis=3)
    mresblock9 = MultiResBlock(32, up9)

    conv10 = keras.layers.Conv2D(1, kernel_size=(1,1), strides=(1,1), padding="same")(mresblock9)
    conv10 = keras.layers.BatchNormalization( scale=False)(conv10)
    conv10= keras.layers.Activation(activation="sigmoid")(conv10)
    
    out = keras.layers.Flatten()(conv10)
    out= keras.layers.Dropout(0.1)(out)
    out= keras.layers.Dense(30)(out)

    model = keras.Model(inputs=[inputs], outputs=[out])

    return model

In [ ]:
input_shape=(96, 96,1)
model =MultiResUnet(input_shape)
model.summary()

In [ ]:
keras.utils.plot_model(model, to_file="MutiResUnet.png", dpi=200)

### The MultiresUnet model was taken from the paper [MultiResUNet : Rethinking the U-Net Architecture for Multimodal Biomedical Image Segmentation](https://www.sciencedirect.com/science/article/abs/pii/S0893608019302503)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=20)
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss',verbose=1, save_best_only=True, save_weights_only=True)


model.compile(optimizer= Adam(), loss='mean_squared_error',  metrics=['mae', 'acc', 'mse'])



augment = albumentations.Compose([albumentations.ShiftScaleRotate(rotate_limit=20, p=0.5),
                           albumentations.RandomBrightnessContrast(p=0.5),
                           albumentations.GaussianBlur(p=0.3),
                           albumentations.GaussNoise(var_limit=(1e-5, 1e-3), p=0.5)],
                          keypoint_params=albumentations.KeypointParams(format='xy', remove_invisible=False))

In [ ]:
train=  data_process(xtrain, ytrain, batch_size=128, augmentation=None)


In [ ]:
history= model.fit(train,   steps_per_epoch=len(train),
                   validation_data=(xval, yval), batch_size=128,
                   epochs=45, verbose=1, callbacks=[early_stop, checkpoint])

In [ ]:
def plot_history(metric):
    plt.plot(history.history[metric])
    plt.plot(history.history["val_{}".format(metric)])
    plt.title('{} vs Epoch'.format(metric))
    plt.ylabel(metric)
    plt.xlabel('Epochs')
    plt.legend(['train', 'validation'], loc='upper right')
    plt.show()


In [ ]:
plot_history("acc")
plot_history("loss")
plot_history("mae")
plot_history("mse")

In [ ]:
model.load_weights('./model.h5')

prediction= model.predict(test_images)

In [ ]:
sample_images(test_images, prediction)

# 

In [ ]:
print("Accuracy: {}".format(np.mean(history.history['acc'])))
print("Loss: {}".format(np.mean(history.history['loss'])))
print("Mean Absolute error: {}".format(np.mean(history.history['mae'])))
print("Mean Squared Error: {}".format(np.mean(history.history['mse'])))

## Accuracy: 0.832591313123703
## Loss: 8.787642423311869
## Mean Absolute error: 0.9814965281221602
## Mean Squared Error: 8.787642423311869